# Adding & Calulating Door Data for 5311 Agencies

* adding door counts using [Eric Dasmalchi's Notebook](https://github.com/cal-itp/data-analyses/blob/572e255f997e7fa0969c88caf23a7791070ad28a/lossan_validators/lossan_validators.ipynb) count for the known 5311 Agencies
 
 
`{'bus':2, 'articulated_bus':3,
'over_the_road_bus':1,
'vintage_historic_trolley':2,
'trolleybus':2,
'van':1,
'cutaway':1, 
'automobile':1,
minivan':1,
'sport_utility_vehicle:':1,
'lrt_stn':6, 'hrt_stn':15,
'la_union_stn':40,
'commuter_rail_stn':6,
'interchange':20, ## interchange is for places like 7th/Metro
'ferry_stn':4})`

In [40]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from calitp import *
from plotnine import *
import intake

In [41]:
from shared_utils import geography_utils

In [2]:
import data_prep

In [3]:
#df = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
cw1 = pd.read_parquet("test_crosswalk_both.parquet")

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/home/jovyan/data-analyses/5311/data_prep.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [4]:
vehicles.sample()

,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups
1366,Fresno County Rural Transit Agency,Fresno,CA,9R02-025,9R02-91007,Independent Public Agency or Authority of Tran...,Rural Reporter,0,77,Bus,NaN,3,0,0,0,0,0,21,4.095238,0.0,18,0,Bus


In [5]:
cw1.head()

,agency_x,ntd_id,itp_id
0,Fresno County Rural Transit Agency,9R02-91007,117.0
1,Eastern Sierra Transit Authority,9R02-91062,99.0
2,Kern Regional Transit,9R02-91059,146.0
3,Lake Transit Authority,9R02-91053,159.0
4,Mendocino Transit Authority,9R02-91047,198.0


In [6]:
cw1 = cw1.rename(columns={"agency_x": "agency"})

In [7]:
df = cw1.merge(vehicles, how="outer", on="agency", indicator=True)

In [8]:
df>>count(_._merge)

,_merge,n
0,left_only,0
1,right_only,23
2,both,119


In [9]:
df>>filter(_._merge=="right_only")>>count(_.agency)

,agency,n
0,Calaveras Transit Agency,3
1,"City of Dixon, dba: Readi-Ride",2
2,"City of Ridgecrest, dba: Ridgerunner",3
3,County of Sacramento Municipal Services Agency...,2
4,"Mariposa County Transit, dba: Mari-Go",4
5,Modoc Transportation Agency,2
6,Palo Verde Valley Transit Agency,1
7,San Benito County LTA,4
8,"Wasco, City of",2


In [10]:
df>>filter(_._merge=="both")>>count(_.agency)>>arrange(-_.n)

,agency,n
23,Fresno County Rural Transit Agency,6
1,Amador Regional Transit System,5
20,Colusa County Transit Agency,5
22,Eastern Sierra Transit Authority,5
32,"Mountain Area Regional Transit Authority, dba:...",5
37,Tehama County,5
2,City of Arcata,4
8,City of Dinuba,4
21,"County of Siskiyou, dba: Siskiyou County Transit",4
25,Humboldt Transit Authority,4


In [67]:
# summary of the vehicle counts

In [68]:
age =geography_utils.aggregate_by_geography(vehicles, 
                       group_cols = ["agency"],
                       sum_cols = ["total_vehicles", "_0_9","_10_12", "_13_15", "_16_20","_21_25","_26_30","_31_60","_60+"],
                       mean_cols = ["average_age_of_fleet__in_years_", "average_lifetime_miles_per_vehicle"], 
                       nunique_cols = ["ntd_id","vehicle_type","vehicle_groups"]
                                      ).sort_values(["agency","total_vehicles"], ascending=[True, True])